# Prédictions : courbes d'apprentissages selon différents modèles et différents paramètres

Afin d'y voir un peu plus clair dans la jungle des paramètres possibles 

 + feature engineering avec différentes échelles
 + différents tactiques d'imputation de données
 + etc
 
 on va tracer des courbes d'apprentissage, afin de sélectionner les meilleures options